# Credit Risk Analysis (Multi-class classification)

## Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os
from statsmodels.tools.tools import add_constant



## Load the dataset

In [2]:
r1=pd.read_excel("case_study1.xlsx")
r2=pd.read_excel("case_study2.xlsx")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7c652c01c250>>
Traceback (most recent call last):
  File "/home/himanshu/PycharmProjects/pythonProject/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
df1=r1.copy()
df2=r2.copy()

## Preprocessing of dataset

In [ ]:
df1.shape

In [ ]:
df2.shape

In [ ]:
df1.sample(5)

In [ ]:
df2.sample(5)

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
df1.describe()

In [ ]:
print(df1["Age_Oldest_TL"].value_counts().get(-99999))
print(df1["Age_Newest_TL"].value_counts().get(-99999))

### In df1 "Age_Oldest_TL", "Age_Newest_TL" have -99999 as a value which denotes NULL value.
### Since the count of these values is only 40 and we have around 52000 rows, I have decided to remove these rows.

In [ ]:
df1=df1.loc[df1['Age_Newest_TL']!=-99999]
df1=df1.loc[df1['Age_Oldest_TL']!=-99999]

In [ ]:
df1.shape

In [ ]:
df2.describe()

### If we closely analyse, we have many columns in df2 which have NULL values indicated by -99999
### We have to check the count for these Null values. Since we have 52000 rows, I am considering if the count of Null values > 10000 (20%), I will drop that column and if less I will just drop the rows with Null values.

In [ ]:
columns_to_be_removed = []

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        columns_to_be_removed .append(i)

df2 = df2.drop(columns_to_be_removed, axis =1)
for i in df2.columns:
    df2 = df2.loc[ df2[i] != -99999 ]

In [ ]:
columns_to_be_removed

### Now since we have 2 datasets, I will combine both datasets with a common column.

In [ ]:
# Checking common column names
for i in list(df1.columns):
    if i in list(df2.columns):
        print (i)

In [ ]:
# Merge the two dataframes, inner join so that no nulls are present
df = pd. merge ( df1, df2, how ='inner', left_on = ['PROSPECTID'], right_on = ['PROSPECTID'] )

### We have our combined dataset now

In [ ]:
df.sample(5)

In [ ]:
df.shape

### Check how many columns are categorical

In [ ]:
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

# EDA

In [ ]:
import matplotlib.pyplot as plt
# Approved_Flag contains 'P1', 'P2', 'P3', 'P4'
flag_mapping = {'P1': 1, 'P2': 2, 'P3': 3, 'P4': 4}
df['Approved_Flag_Num'] = df['Approved_Flag'].map(flag_mapping)

# Set up the matplotlib figure
plt.figure(figsize=(6, 4))

# Create the bar chart
ax = sns.countplot(x='Approved_Flag_Num', data=df, palette='viridis', hue='Approved_Flag', legend=False)

# Adjust x-axis labels to show original categorical values
# Calculate bar width and center the labels
xticks = [tick  for tick in range(len(flag_mapping))]

# Update x-axis labels and positions
ax.set_xticks(xticks)
ax.set_xticklabels(flag_mapping.keys())

# Titles and labels
plt.title('Count of Approved_Flag')
plt.xlabel('Approved_Flag')
plt.ylabel('Count')

# Show plot
plt.show()
df = df.drop(['Approved_Flag_Num'],axis =1)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df is your DataFrame

# Define the order for Approved_Flag categories
approved_flag_order = ['P1', 'P2', 'P3', 'P4']

# List of categorical variables
categorical_vars = ['MARITALSTATUS', 'EDUCATION', 'GENDER']

# Set up the matplotlib figure
plt.figure(figsize=(20, 10))

# Loop through each categorical variable to create a bar plot
for i, var in enumerate(categorical_vars, 1):
    plt.subplot(2, 3, i)
    sns.countplot(x=var, hue='Approved_Flag', data=df, hue_order=approved_flag_order, palette='viridis')
    plt.title(f'Distribution of {var} by Approved_Flag')
    plt.xlabel(var)
    plt.ylabel('Count')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df is your DataFrame

# Define the order for Approved_Flag categories
approved_flag_order = ['P1', 'P2', 'P3', 'P4']

# List of categorical variables to compare against Approved_Flag
categorical_vars = ['MARITALSTATUS', 'EDUCATION', 'GENDER']

# Set up the matplotlib figure
plt.figure(figsize=(20, 10))

# Loop through each Approved_Flag category and plot its distribution for each categorical variable
for i, flag in enumerate(approved_flag_order, 1):
    for j, var in enumerate(categorical_vars, 1):
        plt.subplot(len(approved_flag_order), len(categorical_vars), (i - 1) * len(categorical_vars) + j)
        
        # Filter data for the current Approved_Flag category
        filtered_df = df[df['Approved_Flag'] == flag]
        
        # Plot the distribution of the secondary categorical variable
        sns.countplot(x=var, data=filtered_df, hue=var, palette='viridis', legend=False)
        plt.title(f'{var} for {flag}')
        plt.xlabel(var)
        plt.ylabel('Count')

plt.tight_layout()
plt.show()


### Let us analyse the association of diffferent categorical variables with the target variable Approved_Flag

Since we are comparing categorical VS categorical variable, we will use Chi2 - test of Independence.

HYPOTHESIS:

H0 : There is no significant association between the two categorical variables.

H1 : There is a significant association between the two categorical variables.

TEST CRITERIA:

If p-value < 0.05 (alpha = 5% assumed) --> Reject null hypothesis otherwise fail to reject null hypothesis.

In [ ]:
for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']:
    chi2,p_val, dof,exp=chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i,"--->",p_val)


### Since p-value for all the categorical variables < 0.05 , we will reject the null hypothesis. We will keep all the variables in the dataset.

### Lets test for multicollinearity in the dataset, applying sequential VIF for that.

In [ ]:
numeric_columns = []
for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID']:
        numeric_columns.append(i)

In [ ]:
vif_data = df[numeric_columns]
vif_data

In [ ]:
#VIF sequentially check

# vif_data = df[numeric_columns]
# total_columns = vif_data.shape[1]
# columns_to_be_kept = []
# column_index = 0



# for i in range (0,total_columns):
#     vif_value = variance_inflation_factor(vif_data, column_index)
#     print (column_index,'---',vif_value)
    
    
#     if vif_value <= 10:
#         columns_to_be_kept.append( numeric_columns[i] )
#         column_index = column_index+1
    
#     else:
#         vif_data = vif_data.drop([ numeric_columns[i] ] , axis=1)

In [ ]:
# Function to calculate VIF for each feature
def calculate_vif(X):
    X = add_constant(X)
    vif = pd.DataFrame()
    vif["Feature"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif.drop(index=0)  # Drop the constant (intercept)

# Function to drop all columns with VIF greater than the threshold in one pass
def drop_high_vif_columns_once(X, threshold=8.0):
    vif = calculate_vif(X)
    # Find all features with VIF greater than the threshold
    high_vif_features = vif[vif["VIF"] > threshold]["Feature"].tolist()
    
    if high_vif_features:
        print(f"Dropping features with VIF > {threshold}: {high_vif_features}")
        X = X.drop(columns=high_vif_features)
    
    return X, vif[vif["Feature"].isin(X.columns)]  # Return reduced X and updated VIF

# Drop columns with VIF greater than 10
X_reduced, final_vif = drop_high_vif_columns_once(vif_data, threshold=6.0)

print("\nFinal VIF values:")
final_vif

In [ ]:
columns_to_be_kept = final_vif['Feature']

In [ ]:
X_reduced

In [ ]:
# VIF sequentially check

#vif_data = df[numeric_columns]
# total_columns = vif_data.shape[1]
# columns_to_be_kept = []
# column_index = 0



# for i in range (0,total_columns):
#     vif_value = variance_inflation_factor(vif_data, column_index)
#     print (column_index,'---',vif_value)
    
    
#     if vif_value <= 6:
#         columns_to_be_kept.append( numeric_columns[i] )
#         column_index = column_index+1
    
#     else:
#         vif_data = vif_data.drop([ numeric_columns[i] ] , axis=1)

In [ ]:
# check Anova for columns_to_be_kept 

from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])  
    b = list(df['Approved_Flag'])  
    
    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)


In [ ]:
len(columns_to_be_kept_numerical)

In [ ]:
# listing all the final features
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]

In [ ]:
df

In [ ]:
# Label encoding for the categorical features
['MARITALSTATUS', 'EDUCATION', 'GENDER' , 'last_prod_enq2' ,'first_prod_enq2']



df['MARITALSTATUS'].unique()    
df['EDUCATION'].unique()
df['GENDER'].unique()
df['last_prod_enq2'].unique()
df['first_prod_enq2'].unique()



# Ordinal feature -- EDUCATION
# SSC            : 1
# 12TH           : 2
# GRADUATE       : 3
# UNDER GRADUATE : 3
# POST-GRADUATE  : 4
# OTHERS         : 1
# PROFESSIONAL   : 3


# Others has to be verified by the business end user 




df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']]             = 2
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3




df['EDUCATION'].value_counts()
df['EDUCATION'] = df['EDUCATION'].astype(int)
df.info()





In [ ]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])



df_encoded.info()
k = df_encoded.describe()

In [ ]:
df_encoded

In [ ]:
# Logestic regression
from sklearn.linear_model import LogisticRegression

y = df_encoded['Approved_Flag']
x = df_encoded.drop( ['Approved_Flag'], axis = 1)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
log_reg = LogisticRegression(solver='newton-cg')
log_reg.fit(x_train, y_train)
y_pred = log_reg.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy}')
print ()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()
     

In [ ]:

# Machine Learing model fitting
# Data processing

# 1. Random Forest
rf_classifier = RandomForestClassifier(n_estimators = 200, random_state=42)
rf_classifier.fit(x_train, y_train)
y_pred = rf_classifier.predict(x_test)



accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy}')
print ()
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()
    





In [ ]:
import pandas as pd
import plotly.express as px

# Sample DataFrame
# df = pd.DataFrame({
#     'col_1': [value1, value2, ...],
#     'col_2': [value1, value2, ...],
#     'col_3': [value1, value2, ...],
#     'col_4': [value1, value2, ...],
#     'Approved_Flag': ['P1', 'P2', 'P3', 'P4', ...]
# })

parallel_plot_df = df[['GL_Flag','GENDER','recent_level_of_deliq','Credit_Score','Approved_Flag']]
parallel_plot_df['Approved_Flag'] = parallel_plot_df['Approved_Flag'].astype('category').cat.codes + 1

# Create the parallel coordinates plot
fig = px.parallel_coordinates(
    parallel_plot_df,
    dimensions=['Credit_Score','GENDER','recent_level_of_deliq','GL_Flag'],
    color='Approved_Flag',
    color_continuous_scale=px.colors.sequential.Viridis,
    #labels={'col_1': 'Feature 1', 'col_2': 'Feature 2', 'col_3': 'Feature 3', 'col_4': 'Feature 4'}
)

# Update layout for better readability
fig.update_layout(
    title='Parallel Coordinates Plot',
    coloraxis_colorbar=dict(title='Approved_Flag')
)

# Show the plot
fig.show()


In [ ]:
# 2. xgboost

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)



y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

#Bureau 1         165
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)




xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.5f}')
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


In [ ]:
# Hyperparameter tuning in xgboost
from sklearn.model_selection import GridSearchCV
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

# Define the XGBClassifier with the initial set of hyperparameters
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)

# Define the parameter grid for hyperparameter tuning

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with the best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(x_test, y_test)
print("Test Accuracy:", accuracy)

# Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}


# Based on risk appetite of the bank, you will suggest P1,P2,P3,P4 to the business end user



In [ ]:
df.info()

In [ ]:
r2.describe()

In [ ]:
columns_to_be_removed

In [ ]:
test=r2[columns_to_be_removed]

In [ ]:
for i in test.columns:
    print(i,'-->',test[i].value_counts().get(-99999))

In [ ]:
x

In [ ]:
from kan import *
torch.set_default_dtype(torch.float64)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# create a KAN: 2D inputs, 1D output, and 5 hidden neurons. cubic spline (k=3), 5 grid intervals (grid=5).
model_kan = KAN(width=[42,5,1], grid= 3, k=3 , seed=42, device=device)


In [ ]:
x

In [ ]:
x['Approved_Flag'] = y.map({'P1':1 , 'P2':2, 'P3':3 , 'P4':4})

In [ ]:
x

In [ ]:
x_P1 = x[y == 'P1']
x_P2 = x[y == 'P2']
x_P3 = x[y == 'P3']
x_P4 = x[y == 'P4']

In [ ]:
x1_downsampled = x_P1.sample(200 , random_state= 42)
x2_downsampled = x_P2.sample(200 , random_state= 42)
x3_downsampled = x_P3.sample(200 , random_state= 42)
x4_downsampled = x_P4.sample(200 , random_state= 42)

X_downsampled = pd.concat([x1_downsampled,x2_downsampled,x3_downsampled,x4_downsampled])


In [ ]:
X_downsampled = X_downsampled.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
X = X_downsampled.drop(['Approved_Flag'] , axis =1 )

In [ ]:
X

In [ ]:
y = X_downsampled['Approved_Flag']

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming X and y are already loaded as DataFrames/Series
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Store the splits in a dictionary
X_t = X_train.values
X_t_64 = X_t.astype(np.float64)
X_te = X_test.values
X_te_64 = X_te.astype(np.float64)
dataset = {
    'train_input': torch.tensor(X_t_64, dtype=torch.float64),
    'train_label': torch.tensor(y_train.values, dtype=torch.float64),
    'test_input': torch.tensor(X_te_64, dtype=torch.float64),
    'test_label': torch.tensor(y_test.values, dtype=torch.float64)
}


In [ ]:
dataset['train_input'].shape

In [ ]:
dataset['test_label'].shape 

In [ ]:
model_kan(dataset['train_input'])
model_kan.plot()

In [ ]:
model_kan.train()
model_kan.plot()

In [ ]:
model_kan.prune()
model_kan.plot()